In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import gzip
import json
from transformers import pipeline
import math

In [50]:
raw = pd.read_json('modcloth_final_data.json.gz',lines=True)

In [51]:
raw

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,user_name,length,fit,user_id,shoe size,shoe width,review_summary,review_text
0,123373,29.0,7,5.0,d,38.0,34.0,new,36,5ft 6in,Emily,just right,small,991571,NaN,NaN,NaN,NaN
1,123373,31.0,13,3.0,b,30.0,36.0,new,NaN,5ft 2in,sydneybraden2001,just right,small,587883,NaN,NaN,NaN,NaN
2,123373,30.0,7,2.0,b,NaN,32.0,new,NaN,5ft 7in,Ugggh,slightly long,small,395665,9.0,NaN,NaN,NaN
3,123373,NaN,21,5.0,dd/e,NaN,NaN,new,NaN,NaN,alexmeyer626,just right,fit,875643,NaN,NaN,NaN,NaN
4,123373,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,dberrones1,slightly long,small,944840,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82785,807722,NaN,8,4.0,b,NaN,36.0,outerwear,NaN,5ft 8in,Jennifer,just right,fit,727820,8.5,average,Cute jacket!,Cute jacket!
82786,807722,NaN,12,5.0,ddd/f,NaN,34.0,outerwear,NaN,5ft 5in,Kelli,slightly long,small,197040,NaN,NaN,It's a beautiful jacket.,It's a beautiful jacket. I love how it's knit ...
82787,807722,NaN,12,5.0,dddd/g,36.0,32.0,outerwear,NaN,5ft 4in,elacount,just right,fit,102493,NaN,NaN,I love this blazer. It is,I love this blazer. It is a great office piece...
82788,807722,NaN,12,4.0,NaN,NaN,NaN,outerwear,NaN,5ft 3in,jennaklinner,just right,fit,756491,NaN,NaN,I love this blazer!! I wo,I love this blazer!! I wore it yesterday and g...


In [52]:
sentiment_analysis = pipeline("sentiment-analysis")
def review_text_transform(x):
    if not isinstance(x, str): return 0.5
    if len(x) >= 512 : return 0.5
    label = sentiment_analysis(x)[0]['label']
    if label == 'POSITIVE':
        return 1
    else:
        return 0

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [53]:
for i in range(len(raw['review_text'])):
    raw['review_text'][i] = review_text_transform(raw['review_text'][i])

C:\Users\stanl\AppData\Local\Temp\ipykernel_17192\401756763.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['review_text'][i] = review_text_transform(raw['review_text'][i])


In [ ]:
raw = raw.drop(['user_name','length','review_summary','item_id','user_id'],axis=1)

In [118]:
raw.to_csv('sentiment.csv')

In [121]:
###########################################################

In [2]:
raw = pd.read_csv('sentiment.csv',index_col = False)

c:\software\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
raw = raw.drop(['Unnamed: 0'],axis=1)
raw

,waist,size,quality,cup size,hips,bra size,category,bust,height,fit,shoe size,shoe width,review_text
0,29.0,7,5.0,d,38.0,34.0,new,36.0,5ft 6in,small,NaN,NaN,NaN
1,31.0,13,3.0,b,30.0,36.0,new,NaN,5ft 2in,small,NaN,NaN,NaN
2,30.0,7,2.0,b,NaN,32.0,new,NaN,5ft 7in,small,9.0,NaN,NaN
3,NaN,21,5.0,dd/e,NaN,NaN,new,NaN,NaN,fit,NaN,NaN,NaN
4,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,small,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82785,NaN,8,4.0,b,NaN,36.0,outerwear,NaN,5ft 8in,fit,8.5,average,1.0
82786,NaN,12,5.0,ddd/f,NaN,34.0,outerwear,NaN,5ft 5in,small,NaN,NaN,1.0
82787,NaN,12,5.0,dddd/g,36.0,32.0,outerwear,NaN,5ft 4in,fit,NaN,NaN,1.0
82788,NaN,12,4.0,NaN,NaN,NaN,outerwear,NaN,5ft 3in,fit,NaN,NaN,1.0


In [4]:
raw['waist'].isnull().sum()

79908

In [5]:
raw['size'].isnull().sum()

0

In [6]:
raw['quality'].isnull().sum()

68

In [7]:
raw['cup size'].isnull().sum()

6255

In [8]:
raw['hips'].isnull().sum()

26726

In [9]:
raw['bra size'].isnull().sum()

6018

In [10]:
raw['category'].isnull().sum()

0

In [11]:
raw['bust'].isnull().sum()

70936

In [12]:
raw['height'].isnull().sum()

1107

In [13]:
raw['shoe size'].isnull().sum()

54875

In [14]:
raw['shoe width'].isnull().sum()

64183

In [15]:
raw['cup size'].unique()

array(['d', 'b', 'dd/e', 'c', 'ddd/f', 'dddd/g', 'i', 'a', nan, 'h', 'aa',
       'j', 'k'], dtype=object)

In [16]:
def transform_cup_size(x):
    if not isinstance(x,str):
        return x
    if x == 'd': return 4
    if x == 'b': return 2
    if x == 'dd/e': return 5
    if x == 'c': return 3
    if x == 'ddd/f' : return 6
    if x == 'dddd/g': return 7
    if x == 'i': return 9
    if x == 'a': return 1
    if x == 'h': return 8
    if x == 'aa': return 0.5
    if x == 'j': return 10
    if x == 'k': return 11

In [17]:
raw['cup size'] = raw['cup size'].apply(transform_cup_size)
raw

,waist,size,quality,cup size,hips,bra size,category,bust,height,fit,shoe size,shoe width,review_text
0,29.0,7,5.0,4.0,38.0,34.0,new,36.0,5ft 6in,small,NaN,NaN,NaN
1,31.0,13,3.0,2.0,30.0,36.0,new,NaN,5ft 2in,small,NaN,NaN,NaN
2,30.0,7,2.0,2.0,NaN,32.0,new,NaN,5ft 7in,small,9.0,NaN,NaN
3,NaN,21,5.0,5.0,NaN,NaN,new,NaN,NaN,fit,NaN,NaN,NaN
4,NaN,18,5.0,2.0,NaN,36.0,new,NaN,5ft 2in,small,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82785,NaN,8,4.0,2.0,NaN,36.0,outerwear,NaN,5ft 8in,fit,8.5,average,1.0
82786,NaN,12,5.0,6.0,NaN,34.0,outerwear,NaN,5ft 5in,small,NaN,NaN,1.0
82787,NaN,12,5.0,7.0,36.0,32.0,outerwear,NaN,5ft 4in,fit,NaN,NaN,1.0
82788,NaN,12,4.0,NaN,NaN,NaN,outerwear,NaN,5ft 3in,fit,NaN,NaN,1.0


In [18]:
def transform_height(x):
    if not isinstance(x,str):
        return x
    x = x.split(' ')
    feet = int(x[0][:-2])
    if len(x) == 2:
        inch = int(x[1][:-2])
    else:
        inch = 0
    cm = (30.48 * feet) + (2.54 * inch)
    return cm

In [19]:
raw['height'] = raw['height'].apply(transform_height)
raw

,waist,size,quality,cup size,hips,bra size,category,bust,height,fit,shoe size,shoe width,review_text
0,29.0,7,5.0,4.0,38.0,34.0,new,36.0,167.64,small,NaN,NaN,NaN
1,31.0,13,3.0,2.0,30.0,36.0,new,NaN,157.48,small,NaN,NaN,NaN
2,30.0,7,2.0,2.0,NaN,32.0,new,NaN,170.18,small,9.0,NaN,NaN
3,NaN,21,5.0,5.0,NaN,NaN,new,NaN,NaN,fit,NaN,NaN,NaN
4,NaN,18,5.0,2.0,NaN,36.0,new,NaN,157.48,small,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82785,NaN,8,4.0,2.0,NaN,36.0,outerwear,NaN,172.72,fit,8.5,average,1.0
82786,NaN,12,5.0,6.0,NaN,34.0,outerwear,NaN,165.10,small,NaN,NaN,1.0
82787,NaN,12,5.0,7.0,36.0,32.0,outerwear,NaN,162.56,fit,NaN,NaN,1.0
82788,NaN,12,4.0,NaN,NaN,NaN,outerwear,NaN,160.02,fit,NaN,NaN,1.0


In [20]:
raw['fit'].unique()

array(['small', 'fit', 'large'], dtype=object)

In [21]:
def transform_fit(x):
    if x == 'fit':
        return 1
    else:
        return 0

In [22]:
raw['fit'] = raw['fit'].apply(transform_fit)
raw

,waist,size,quality,cup size,hips,bra size,category,bust,height,fit,shoe size,shoe width,review_text
0,29.0,7,5.0,4.0,38.0,34.0,new,36.0,167.64,0,NaN,NaN,NaN
1,31.0,13,3.0,2.0,30.0,36.0,new,NaN,157.48,0,NaN,NaN,NaN
2,30.0,7,2.0,2.0,NaN,32.0,new,NaN,170.18,0,9.0,NaN,NaN
3,NaN,21,5.0,5.0,NaN,NaN,new,NaN,NaN,1,NaN,NaN,NaN
4,NaN,18,5.0,2.0,NaN,36.0,new,NaN,157.48,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82785,NaN,8,4.0,2.0,NaN,36.0,outerwear,NaN,172.72,1,8.5,average,1.0
82786,NaN,12,5.0,6.0,NaN,34.0,outerwear,NaN,165.10,0,NaN,NaN,1.0
82787,NaN,12,5.0,7.0,36.0,32.0,outerwear,NaN,162.56,1,NaN,NaN,1.0
82788,NaN,12,4.0,NaN,NaN,NaN,outerwear,NaN,160.02,1,NaN,NaN,1.0


In [23]:
raw['shoe width'].unique()

array([nan, 'wide', 'average', 'narrow'], dtype=object)

In [24]:
def transform_shoe_width(x):
    if pd.isnull(x) == True:
        return x
    elif x == 'wide' : return 3
    elif x == 'average' : return 2
    elif x == 'narrow' : return 1

In [25]:
raw['shoe width'] = raw['shoe width'].apply(transform_shoe_width)
raw

,waist,size,quality,cup size,hips,bra size,category,bust,height,fit,shoe size,shoe width,review_text
0,29.0,7,5.0,4.0,38.0,34.0,new,36.0,167.64,0,NaN,NaN,NaN
1,31.0,13,3.0,2.0,30.0,36.0,new,NaN,157.48,0,NaN,NaN,NaN
2,30.0,7,2.0,2.0,NaN,32.0,new,NaN,170.18,0,9.0,NaN,NaN
3,NaN,21,5.0,5.0,NaN,NaN,new,NaN,NaN,1,NaN,NaN,NaN
4,NaN,18,5.0,2.0,NaN,36.0,new,NaN,157.48,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82785,NaN,8,4.0,2.0,NaN,36.0,outerwear,NaN,172.72,1,8.5,2.0,1.0
82786,NaN,12,5.0,6.0,NaN,34.0,outerwear,NaN,165.10,0,NaN,NaN,1.0
82787,NaN,12,5.0,7.0,36.0,32.0,outerwear,NaN,162.56,1,NaN,NaN,1.0
82788,NaN,12,4.0,NaN,NaN,NaN,outerwear,NaN,160.02,1,NaN,NaN,1.0


In [26]:
raw['category'].unique()

array(['new', 'dresses', 'wedding', 'sale', 'tops', 'bottoms',
       'outerwear'], dtype=object)

In [27]:
category_one_hot = pd.get_dummies(raw['category'])

In [28]:
raw = raw.drop('category',axis = 1)
raw = raw.join(category_one_hot)

In [29]:
raw

,waist,size,quality,cup size,hips,bra size,bust,height,fit,shoe size,shoe width,review_text,bottoms,dresses,new,outerwear,sale,tops,wedding
0,29.0,7,5.0,4.0,38.0,34.0,36.0,167.64,0,NaN,NaN,NaN,0,0,1,0,0,0,0
1,31.0,13,3.0,2.0,30.0,36.0,NaN,157.48,0,NaN,NaN,NaN,0,0,1,0,0,0,0
2,30.0,7,2.0,2.0,NaN,32.0,NaN,170.18,0,9.0,NaN,NaN,0,0,1,0,0,0,0
3,NaN,21,5.0,5.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,0,0,1,0,0,0,0
4,NaN,18,5.0,2.0,NaN,36.0,NaN,157.48,0,NaN,NaN,NaN,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82785,NaN,8,4.0,2.0,NaN,36.0,NaN,172.72,1,8.5,2.0,1.0,0,0,0,1,0,0,0
82786,NaN,12,5.0,6.0,NaN,34.0,NaN,165.10,0,NaN,NaN,1.0,0,0,0,1,0,0,0
82787,NaN,12,5.0,7.0,36.0,32.0,NaN,162.56,1,NaN,NaN,1.0,0,0,0,1,0,0,0
82788,NaN,12,4.0,NaN,NaN,NaN,NaN,160.02,1,NaN,NaN,1.0,0,0,0,1,0,0,0


In [32]:
raw['bust'].unique()

array(['36.0', nan, '39.0', '34.0', '43.0', '35.0', '38.0', '33.0',
       '40.0', '41.0', '44.0', '45.0', '37.0', '30.0', '32.0', '42.0',
       '48.0', '46.0', '51.0', '25.0', '31.0', '26.0', '52.0', '22.0',
       '50.0', '47.0', '56.0', '58.0', '49.0', '54.0', '55.0', '53.0',
       '27.0', '28.0', '29.0', '20.0', '57.0', '23.0', '44', '42', '34',
       '36', '32', '38', '40', '33', '47', '35', '41', '37', '39', '46',
       '30', '31', '20', '43', '48', '45', '23', '49', '52', '26', '27',
       '53', '28', '56', '57', '37-39', '54', '50', '25', '51', '29',
       '58', '24', '55', 46.0, 34.0, 36.0, 42.0, 48.0, 43.0, 38.0, 37.0,
       32.0, 39.0, 33.0, 35.0, 45.0, 49.0, 44.0, 40.0, 20.0, 41.0, 47.0,
       52.0, 30.0, 23.0, 50.0, 29.0, 54.0, 53.0, 51.0, 25.0, 31.0, 28.0,
       26.0, 55.0, 59.0, 56.0, 24.0], dtype=object)

In [34]:
sum(raw['bust'] == '37-39')

1

In [35]:
def transform_bust(x):
    if pd.isnull(x) == True:
        return x
    if isinstance(x,float): return x
    elif isinstance(x,str):
        if x == '37-39':
            return 38
        else:
            return float(x)

In [36]:
raw['bust'] = raw['bust'].apply(transform_bust)
raw

,waist,size,quality,cup size,hips,bra size,bust,height,fit,shoe size,shoe width,review_text,bottoms,dresses,new,outerwear,sale,tops,wedding
0,29.0,7,5.0,4.0,38.0,34.0,36.0,167.64,0,NaN,NaN,NaN,0,0,1,0,0,0,0
1,31.0,13,3.0,2.0,30.0,36.0,NaN,157.48,0,NaN,NaN,NaN,0,0,1,0,0,0,0
2,30.0,7,2.0,2.0,NaN,32.0,NaN,170.18,0,9.0,NaN,NaN,0,0,1,0,0,0,0
3,NaN,21,5.0,5.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,0,0,1,0,0,0,0
4,NaN,18,5.0,2.0,NaN,36.0,NaN,157.48,0,NaN,NaN,NaN,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82785,NaN,8,4.0,2.0,NaN,36.0,NaN,172.72,1,8.5,2.0,1.0,0,0,0,1,0,0,0
82786,NaN,12,5.0,6.0,NaN,34.0,NaN,165.10,0,NaN,NaN,1.0,0,0,0,1,0,0,0
82787,NaN,12,5.0,7.0,36.0,32.0,NaN,162.56,1,NaN,NaN,1.0,0,0,0,1,0,0,0
82788,NaN,12,4.0,NaN,NaN,NaN,NaN,160.02,1,NaN,NaN,1.0,0,0,0,1,0,0,0


In [39]:
raw.to_csv('transformed.csv')